In [1]:
import json
from StringIO import StringIO
import pandas as pd

In [2]:
import pandas as pd
import os
import sklearn as sk
from sklearn.linear_model import MultiTaskLasso, MultiTaskElasticNet, ElasticNet
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import sklearn.metrics as skmet
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords, names
import datetime
from scipy.stats import ttest_ind 

In [3]:
listings = pd.read_csv('inside_airbnb/listings.csv')
list_summ = pd.read_csv('inside_airbnb/listings_summ.csv')
neighborhoods = pd.read_csv('inside_airbnb/neighbourhoods.csv')
reviews = pd.read_csv('inside_airbnb/reviews.csv')
reviews_summ = pd.read_csv('inside_airbnb/reviews_summ.csv')
calendar = pd.read_csv('inside_airbnb/calendar.csv')

In [5]:
special_events = pd.read_csv('Datasources/other general/special_events_list_datasd.csv')
active_businesses = pd.read_csv('Datasources/other general/sd_active_businesses_datasd.csv')

In [6]:
special_events

,event_title,event_id,event_subtitle,event_type,event_desc,event_loc,event_start,event_end,exp_attendance,exp_participants,event_host,event_url,event_address,latitude,longitude
0,2018 Hillcrest Farmers' Market (Sundays),49953,NaN,FARMERS,This farmer's market in Hillcrest offers an ar...,DMV parking lot at 3690 Normal Street ~ Normal...,2018-12-30 09:00:00,2018-12-30 14:00:00,"2,000",400,Hillcrest Business Association,www.hillcrestfarmersmarket.com,3690 Normal Street,32.748542,-117.149901
1,Old Town Artisan's Market,50424,NaN,FARMERS,A weekend open air market offering an array of...,Harney Street between San Diego Avenue and Con...,2018-12-30 09:00:00,2018-12-30 16:30:00,"1,000",100,Old Town San Diego Chamber of Commerce,NaN,Harney Street & San Diego Avenue,32.752779,-117.194902
2,Sunday Artisan Market,50588,NaN,FARMERS,The Sunday Artisan Market provides space for l...,5th Avenue between Market Street and Island Av...,2018-12-30 09:00:00,2018-12-30 14:00:00,"3,000",20,Gaslamp Quarter Association,www.gaslamp.org,5th Avenue & Market Street,32.731392,-117.160285
3,Old Town Artisan's Market,50423,NaN,FARMERS,A weekend open air market offering an array of...,Harney Street between San Diego Avenue and Con...,2018-12-29 09:00:00,2018-12-29 16:30:00,"1,000",100,Old Town San Diego Chamber of Commerce,NaN,Harney Street & San Diego Avenue,32.752779,-117.194902
4,City Heights Farmer's Market (Every Saturday),50130,NaN,FARMERS,This farmer's market in City Heights offers an...,Wightman Street between 43rd Street and Fairmo...,2018-12-29 09:00:00,2018-12-29 13:00:00,500,100,San Diego County Farm Bureau,www.sdfarmbureau.org,Wightman & 43rd Streets,32.747753,-117.099983
5,The Mercato Little Italy's Farmers' Market,50028,NaN,FARMERS,The Little Italy Mercato is a Certified Farmer...,W. Cedar St between Front St. and Kettner Blvd...,2018-12-29 08:00:00,2018-12-29 14:00:00,"1,500",400,Little Italy Association of San Diego,www.littleitalymercato.com,Cedar & Front Streets,32.721978,-117.161570
6,Holiday Bowl 5K,49893,NaN,ATHLETIC,This nationally televised 5K run/walk is a kic...,Harbor Dr./Ash St. - Harbor Dr. - walkway - Em...,2018-12-28 09:45:00,2018-12-28 12:00:00,"1,000",100,KOZ Events,www.kozenterprises.com,Harbor Drive & Ash Streeet,32.709384,-117.162865
7,North Park Thursday Market,50078,NaN,FARMERS,This farmer's market in North Park offers loca...,North Park Way between Granada Avenue and 30th...,2018-12-27 15:00:00,2018-12-27 19:30:00,900,70,North Park Main Street,www.northparkmarket.com,North Park Way & Granada Avenue,32.747410,-117.126621
8,Ocean Beach Certified Farmer's Market (Wednesd...,50212,NaN,FARMERS,This farmer's market in Ocean Beach offers an ...,Newport Avenue between Bacon and Cable Streets,2018-12-26 16:00:00,2018-12-26 20:00:00,"1,200",60,Ocean Beach Merchant's Association,www.oceanbeachsandiego.com,Newport Avenue & Bacon Street,32.743133,-117.244998
9,Old Town Artisan's Market,50422,NaN,FARMERS,A weekend open air market offering an array of...,Harney Street between San Diego Avenue and Con...,2018-12-23 09:00:00,2018-12-23 16:30:00,"1,000",100,Old Town San Diego Chamber of Commerce,NaN,Harney Street & San Diego Avenue,32.752779,-117.194902


In [7]:
active_businesses

,account_key,address_str,apt_suite,bus_start_dt,business_owner_name,business_phone,city,creation_dt,doing_bus_as_name,expiration_dt,lat,lon,naics_code,naics_description,ownership_type,pmb_box,po_box,state,zip,BID
0,2017014278,10640 scripps ranch blvd,101,2017-08-23,miriam chor freitas,(858) 217-5770,san diego,2017-09-27,miriam chor freitas lcsw,2018-08-31,32.911936,-117.110321,62133,offices of other mental health practitioners,sole,NaN,NaN,CA,92131-1095,NaN
1,2017013866,5085 september st,NaN,2015-09-01,robert burke,NaN,san diego,2017-09-27,robert burke,2018-08-31,32.780080,-117.187049,541,"professional, scientific & technical services",sole,NaN,NaN,CA,92110-1222,NaN
2,2017013873,827 beryl st,NaN,2015-09-01,robert doll,NaN,san diego,2017-09-27,robert doll,2018-08-31,32.803056,-117.256734,541,"professional, scientific & technical services",sole,NaN,NaN,CA,92109-2004,NaN
3,2017013871,2704 boston ave,NaN,2015-09-01,jose m espinozanaranjo & maria m dadaespinoza,NaN,san diego,2017-09-27,jose m espinozanaranjo & maria m dadaespinoza,2018-08-31,32.694014,-117.135811,541,"professional, scientific & technical services",h-w,NaN,NaN,CA,92113-3708,NaN
4,2017013870,1502 mesa brook st,NaN,2015-09-01,feliz h blanco & idalia contreras ochoa,NaN,san diego,2017-09-27,feliz h blanco & idalia contreras ochoa,2018-08-31,32.693291,-117.048771,541,"professional, scientific & technical services",h-w,NaN,NaN,CA,92114-7875,NaN
5,2017013869,4775 monongahela st,NaN,2015-09-01,paul v saadieh,NaN,san diego,2017-09-27,paul v saadieh,2018-08-31,32.834477,-117.228387,541,"professional, scientific & technical services",sole,NaN,NaN,CA,92117-2419,NaN
6,2017013867,302 washington st,150-31,2015-09-01,scott corich,NaN,san diego,2017-09-27,scott corich,2018-08-31,32.750293,-117.162618,541,"professional, scientific & technical services",sole,NaN,NaN,CA,92103-2110,Hillcrest
7,2017013865,2464 fenton pkwy,207,2015-09-01,eduardo d galvan & nallely p padilla,NaN,san diego,2017-09-27,eduardo d galvan & nallely p padilla,2018-08-31,32.780464,-117.131484,541,"professional, scientific & technical services",h-w,NaN,NaN,CA,92108-6705,NaN
8,2017013874,2616 highland ave,5,2015-09-01,ketkeo inthavong,NaN,san diego,2017-09-27,ketkeo inthavong,2018-08-31,32.733630,-117.098926,541,"professional, scientific & technical services",sole,NaN,NaN,CA,92105-4371,NaN
9,2017013863,4584 f st,NaN,2015-09-01,clarence e davis sr,NaN,san diego,2017-09-27,clarence e davis sr,2018-08-31,32.713745,-117.095833,541,"professional, scientific & technical services",sole,NaN,NaN,CA,92102-3664,NaN
